In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Dataset: https://github.com/owid/energy-data?tab=readme-ov-file

ONLY europe!

In [53]:
energy = pd.read_csv('data/owid-energy-data.csv')

In [54]:
consumption = ['country', 'year', 'iso_code', 'population', 'gdp',
 'wind_consumption',
 'solar_consumption',
 'renewables_consumption',
 'other_renewable_consumption',
 'oil_consumption',
 'nuclear_consumption',
 'low_carbon_consumption',
 'hydro_consumption',
 'gas_consumption',
 'fossil_fuel_consumption',
 'coal_consumption',
 'biofuel_consumption',]

In [55]:
energy = energy[energy['year'] == 2019]

In [56]:
# keep only the columns we need
energy = energy[consumption]
energy.dropna(inplace=True)

In [57]:
share = energy.drop(columns=['country', 'year', 'iso_code', 'population', 'gdp'])
share.index = energy['country']
share = share.div(share.sum(axis=1), axis=0)

In [58]:
mean_shares = share.mean()

In [59]:
mean_shares.sort_values(ascending=False, inplace=True)
mean_shares

fossil_fuel_consumption        0.366096
oil_consumption                0.176918
gas_consumption                0.105873
low_carbon_consumption         0.097787
coal_consumption               0.083306
renewables_consumption         0.072233
hydro_consumption              0.038412
nuclear_consumption            0.025554
wind_consumption               0.013771
other_renewable_consumption    0.011042
solar_consumption              0.004725
biofuel_consumption            0.004283
dtype: float64

In [60]:
HHI = (mean_shares ** 2).sum()
effective_N = 1 / HHI
print(f"Herfindahl index: {HHI:.3f}, Effective shocks: {effective_N:.0f}")


Herfindahl index: 0.201, Effective shocks: 5


In [61]:
share.to_csv('data/shares.csv')

In [62]:
energy_price_shock = {
    "fossil_fuel_consumption": 0.539,       # +53.9%
    "oil_consumption": 0.329,               # +32.9%
    "gas_consumption": 0.934,               # +93.4%
    "coal_consumption": 1.234,              # +123.4%
    "low_carbon_consumption": 0.0,          # no traded fuel
    "renewables_consumption": 0.0,          # no traded fuel
    "hydro_consumption": 0.0,               # no traded fuel
    "nuclear_consumption": 0.102,           # +10.2%
    "wind_consumption": 0.0,                # no traded fuel
    "other_renewable_consumption": 0.0,     # no traded fuel
    "solar_consumption": 0.0,               # no traded fuel
    "biofuel_consumption": 0.0              # fragmented, treated as 0
}


In [63]:
# Step 3: Ensure all columns in the price shock dict are in the dataframe
common_cols = [col for col in energy_price_shock if col in share.columns]

# Step 4: Compute the Bartik IV as the dot product of shares × global shocks
share["bartik_iv"] = share[common_cols].dot(pd.Series(energy_price_shock))


In [67]:
bartik_iv = share["bartik_iv"]

bartik_iv.sort_values(ascending=False)
bartik_iv.to_csv('data/bartik_iv.csv')